# **Modelamiento**

## Librerías

In [1]:
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3 MB 58.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1


In [2]:
!pip install unidecode

     |████████████████████████████████| 235 kB 30.0 MB/s 


In [3]:
#Generales
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Proceso de Modelación
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error,r2_score,median_absolute_error, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, ShuffleSplit

## Lectura Información

In [4]:
#Descargar datasets desde github
!git clone https://github.com/andres-soto-h/monografia-udea-eacd.git

Cloning into 'monografia-udea-eacd'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [5]:
#Lectura del dataset transformado
df_propiedades=pd.read_csv('/content/monografia-udea-eacd/df_prop_clean_12082021.csv', delimiter=';', encoding='latin1')

In [6]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd '/content/drive/My Drive/Colab Notebooks/Seminario/DATASETS'
# # !ls
# df_propiedades=pd.read_csv('df_prop_clean_12082021.csv', delimiter=';', encoding='latin1') 
# print(df_propiedades.shape)

In [7]:
df_propiedades.rename(columns={"municipio":"ubicacion"}, inplace=True)

**NOTA:** se decide eliminar la información del municipio de **La Unión**.

In [8]:
mask_union = df_propiedades['ubicacion']!='la uniÃ³n'
df_propiedades = df_propiedades[mask_union]

In [9]:
df_propiedades['ubicacion'].value_counts()

rionegro                  1241
el retiro                  439
la ceja                    342
guarne                     196
marinilla                  117
el carmen de viboral        62
san antonio de pereira      38
llanogrande                 31
santuario                   15
Name: ubicacion, dtype: int64

**Nota2:** Se convierte la variable Estrato en ordinal

In [10]:
df_propiedades['estrato'] = df_propiedades['estrato'].replace('Campestre', '0')
df_propiedades['estrato'] = df_propiedades['estrato'].astype(int)
df_propiedades['estrato'].value_counts()

4    947
5    636
3    584
0    162
6     89
2     58
1      5
Name: estrato, dtype: int64

## **Ajuste del Modelo**

**Funciones**

In [11]:
#Métricas datos transformados
def median_absolute_error2(y_true, y_pred):
  return median_absolute_error(np.expm1(y_true), np.expm1(y_pred))

def mean_squared_error2(y_true, y_pred):
  return mean_squared_error(np.expm1(y_true), np.expm1(y_pred), squared=False)


def mean_absolute_percentage_error2(y_true, y_pred):
  return mean_absolute_percentage_error(np.expm1(y_true), np.expm1(y_pred))
  
def r2_score2(y_true, y_pred):
  return r2_score(np.expm1(y_true), np.expm1(y_pred))

In [12]:
def div_train_test(X, y, var_stratify = ''):
  
  if var_stratify != '': 
    x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.9, stratify = X[var_stratify],  random_state = 42)

    var_stratify_train = x_train[var_stratify]
    #ubicacion_test = X_test['ubicacion']

    x_train.drop(var_stratify,axis=1, inplace=True)
    x_test.drop(var_stratify,axis=1, inplace=True)

    #Variable Objetivo Transformada
    y_train_t = np.log1p(y_train)
    y_test_t = np.log1p(y_test)

    return x_train, x_test, y_train, y_test, y_train_t, y_test_t, var_stratify_train

  else:
    x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.9,  random_state = 42)

    #Variable Objetivo Transformada
    y_train_t = np.log1p(y_train)
    y_test_t = np.log1p(y_test)
  
  return x_train, x_test, y_train, y_test, y_train_t, y_test_t

def busqueda_hiperparametros(x_train, y_train, var_stratify_train, estimator,param_grid = [] , scoring_med = 'neg_median_absolute_error', transf=False):

  if len(var_stratify_train) != 0:
    
    print('Seccion Stratify')
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)
    stratified = sss.split(x_train, var_stratify_train)
    
    if transf: 
      print('**para datos transformados**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error2), 'r2':make_scorer(r2_score2) , 'rmse':make_scorer(mean_squared_error2), 'mape':make_scorer(mean_absolute_percentage_error2)}
    else:
      print('**para datos sin transformar**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error), 'r2':make_scorer(r2_score) , 'rmse':make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
    
    grid = GridSearchCV(estimator, param_grid=param_grid, cv = stratified, scoring = scoring_grid , refit = 'metrica',  return_train_score=True, n_jobs=-1, verbose=8)
    return grid.fit(x_train, y_train)
  
  else:
    print('OutStratify')
    
    if transf: 
      print('**para datos transformados**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error2), 'r2':make_scorer(r2_score2) , 'rmse':make_scorer(mean_squared_error2), 'mape':make_scorer(mean_absolute_percentage_error2)}
    else:
      print('**para datos sin transformar**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error), 'r2':make_scorer(r2_score) , 'rmse':make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
    
    grid = GridSearchCV(estimator, param_grid=param_grid, scoring = scoring_grid, refit = 'metrica', return_train_score=True, n_jobs=-1, verbose=8 )
    return grid.fit(X_train, y_train)

def metricas(model,y_train, p_train, y_test, p_test): 

  resultados = {
      'parametros' : [model.get_params()['steps'][1][1]],
      'MeAE_train' : [median_absolute_error(y_train,p_train)],
      'MeAE_test'  : [median_absolute_error(y_test, p_test)],
      'MAPE_train' : [mean_absolute_percentage_error(y_train,p_train)],
      'MAPE_test'  : [mean_absolute_percentage_error(y_test, p_test)],
      'r2_train'   : [r2_score(y_train,p_train)],
      'r2_test'    : [r2_score(y_test, p_test)],
      'rmse_train' : [mean_squared_error(y_train,p_train,squared = False)],
      'rmse_test'  : [mean_squared_error(y_test,p_test,squared = False)]
  }
  return pd.DataFrame(resultados)

**División Covariables  y Variable Objetivo**

In [13]:
data_model=df_propiedades.copy()
#columnas_quitar=['tipo','url','titulo','descripcion','caractint','caractext','caractsec']
# varibles_considerar = ['tipo','precio','area_m2','banos','garajes','antiguedad','latitud','longitud','estrato','ubicacion','tipo_propiedad','balcon','zonas_verdes','en_conjunto_cerrado','zona_infantil','supermercados_ccomerciales','colegios_universidades','trans_publico_cercano']
varibles_considerar = ['tipo','precio','area_m2','banos','garajes','antiguedad','estrato','ubicacion','tipo_propiedad','balcon','zonas_verdes','en_conjunto_cerrado','zona_infantil','supermercados_ccomerciales','colegios_universidades','trans_publico_cercano']

# columnas_quitar=['url','titulo','descripcion','caractint','caractext','caractsec','otros_datos']
# data_model.drop(columnas_quitar, axis=1, inplace=True)
data_model = data_model.loc[:,varibles_considerar]
data_model.shape

(2481, 16)

In [14]:
data_model.head()

,tipo,precio,area_m2,banos,garajes,antiguedad,estrato,ubicacion,tipo_propiedad,balcon,zonas_verdes,en_conjunto_cerrado,zona_infantil,supermercados_ccomerciales,colegios_universidades,trans_publico_cercano
0,Nueva,5.122629e+08,89.49,3.0,1.0,0,0,rionegro,apartamento,1.0,1.0,1.0,1.0,1.0,0.0,1.0
1,Usada,5.340000e+08,182.00,4.0,2.0,1 a 8 aÃ±os,4,la ceja,casa,1.0,0.0,0.0,1.0,1.0,0.0,1.0
2,Usada,2.700000e+08,4070.00,1.0,9.0,1 a 8 aÃ±os,2,guarne,finca,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Usada,1.389622e+09,274.00,5.0,0.0,16 a 30 aÃ±os,1,rionegro,finca,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Usada,1.990000e+08,49.00,2.0,1.0,0,4,el retiro,apartamento,0.0,0.0,1.0,1.0,1.0,0.0,1.0


In [15]:
X = data_model.drop(['precio'], axis=1)
y = data_model['precio']

**OneHotEncoder Variables Categóricas**

In [16]:
enc = OneHotEncoder(handle_unknown='ignore')
data_aux = pd.DataFrame(enc.fit_transform(X[['tipo','tipo_propiedad','ubicacion','antiguedad']]).toarray(), columns= enc.get_feature_names(['tipo','tipo_propiedad','ubicacion','antiguedad']))
data_aux['fila'] = range(0, X.shape[0])
data_aux.head()

X.drop(['tipo','tipo_propiedad','antiguedad'],axis=1,inplace=True)
X['fila'] = range(0, X.shape[0])

X = pd.merge(data_aux,X, on = 'fila', how='inner')
X.drop(['fila'], axis=1, inplace=True)
X.shape

(2481, 33)

**División train y Test**

In [17]:
X_train, X_test, Y_train, Y_test, Y_train_t, Y_test_t, var_stratify_train = div_train_test(X, y, var_stratify = 'ubicacion')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### **Random Forest**

In [ ]:
pipe = Pipeline(steps = [('scaler', StandardScaler()), ('rf', RandomForestRegressor(random_state=42))])

####**Sin transformar la Y**

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'rf__n_estimators':[80,100,120,150], 'rf__max_depth':[3,5,10,20,25], 'rf__min_samples_split':[2,3,5,10,15],'rf__min_samples_leaf':[1,2,5,8]}
para_grid = {'rf__n_estimators':[100,120,150], 'rf__max_depth':[6,7,8,9], 'rf__min_samples_split':[4,5,6], 'rf__min_samples_leaf':[2,3,4]}

modelo_rf = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe, param_grid = para_grid, transf=False)
modelo_rf

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=<generator object BaseShuffleSplit.split at 0x7fb6da75a3d0>,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [6, 7, 8, 9],
                         'rf__min_samples_leaf': [2, 3, 4],
                         'rf__min_samples_split': [4, 5, 6],
                         'rf__n_estimators': [100, 120, 150]},
             refit='metrica', return_train_score=True,
             scoring={'mape': make_scorer(mean_absolute_percentage_error),
                      'meae': make_scorer(median_absolute_error),
                      'metrica': 'neg_median_absolute_error',
                      'r2': make_scorer(r2_score),
                      'rmse': make_scorer(mean_squared_error)},
             verbose=8)

In [ ]:
metricas_rf = metricas(modelo_rf.best_estimator_,Y_train, modelo_rf.predict(X_train), Y_test, modelo_rf.predict(X_test))
metricas_rf

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.926453e+07,7.321584e+07,0.172622,0.215121,0.885647,0.778355,1.908200e+08,3.001326e+08


In [ ]:
modelo_rf.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestRegressor(max_depth=9, min_samples_leaf=3,
                                       min_samples_split=4, n_estimators=150,
                                       random_state=42))])

In [ ]:
dd=pd.DataFrame(modelo_rf.cv_results_)
dd[dd['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica',
'mean_test_meae', 'std_test_meae', 'rank_test_meae','mean_train_meae','std_train_meae',
'mean_test_r2', 'std_test_r2','rank_test_r2','mean_train_r2', 'std_train_r2',
'mean_test_rmse','std_test_rmse', 'rank_test_rmse','mean_train_rmse', 'std_train_rmse',
'mean_test_mape','std_test_mape', 'rank_test_mape', 'mean_train_mape', 'std_train_mape']]

,params,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_test_meae,std_test_meae,rank_test_meae,mean_train_meae,std_train_meae,mean_test_r2,std_test_r2,rank_test_r2,mean_train_r2,std_train_r2,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_train_rmse,std_train_rmse,mean_test_mape,std_test_mape,rank_test_mape,mean_train_mape,std_train_mape
92,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 3...",-7.614567e+07,4.939623e+06,-5.863940e+07,772135.365423,7.614567e+07,4.939623e+06,106,5.863940e+07,772135.365423,0.718674,0.034645,36,0.889917,0.002912,9.546208e+16,1.070703e+16,70,3.473990e+16,8.676187e+14,0.238447,0.009312,91,0.16853,0.001707
95,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 3...",-7.614567e+07,4.939623e+06,-5.863940e+07,772135.365423,7.614567e+07,4.939623e+06,106,5.863940e+07,772135.365423,0.718674,0.034645,36,0.889917,0.002912,9.546208e+16,1.070703e+16,70,3.473990e+16,8.676187e+14,0.238447,0.009312,91,0.16853,0.001707
98,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 3...",-7.614567e+07,4.939623e+06,-5.863940e+07,772135.365423,7.614567e+07,4.939623e+06,106,5.863940e+07,772135.365423,0.718674,0.034645,36,0.889917,0.002912,9.546208e+16,1.070703e+16,70,3.473990e+16,8.676187e+14,0.238447,0.009312,91,0.16853,0.001707


####**Usando Y transformada**

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'rf__n_estimators':[80,100,120,150], 'rf__max_depth':[3,5,10,20,25], 'rf__min_samples_split':[2,3,5,10,15],'rf__min_samples_leaf':[1,2,5,8]}
para_grid = {'rf__n_estimators':[100,120,150], 'rf__max_depth':[6,7,8,9], 'rf__min_samples_split':[4,5,6], 'rf__min_samples_leaf':[2,3,4]}

modelo_rf_t = busqueda_hiperparametros(X_train, Y_train_t,var_stratify_train, pipe, param_grid = para_grid, transf = True)
modelo_rf_t

Seccion Stratify
**para datos transformados**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=<generator object BaseShuffleSplit.split at 0x7fb6d9ed9a50>,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [6, 7, 8, 9],
                         'rf__min_samples_leaf': [2, 3, 4],
                         'rf__min_samples_split': [4, 5, 6],
                         'rf__n_estimators': [100, 120, 150]},
             refit='metrica', return_train_score=True,
             scoring={'mape': make_scorer(mean_absolute_percentage_error2),
                      'meae': make_scorer(median_absolute_error2),
                      'metrica': 'neg_median_absolute_error',
                      'r2': make_scorer(r2_score2),
                      'rmse': make_scorer(mean_squared_error2)},
             verbose=8)

In [ ]:
metricas_rf_t = metricas(modelo_rf_t.best_estimator_,Y_train, np.expm1(modelo_rf_t.predict(X_train)), Y_test, np.expm1(modelo_rf_t.predict(X_test)))
metricas_rf_t

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.295492e+07,7.255825e+07,0.150728,0.20504,0.867465,0.754846,2.054302e+08,3.156481e+08


In [ ]:
modelo_rf_t.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestRegressor(max_depth=9, min_samples_leaf=2,
                                       min_samples_split=5, random_state=42))])

In [ ]:
dd_t=pd.DataFrame(modelo_rf_t.cv_results_)
dd_t[dd_t['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica',
'mean_test_meae', 'std_test_meae', 'rank_test_meae','mean_train_meae','std_train_meae',
'mean_test_r2', 'std_test_r2','rank_test_r2','mean_train_r2', 'std_train_r2',
'mean_test_rmse','std_test_rmse', 'rank_test_rmse','mean_train_rmse', 'std_train_rmse',
'mean_test_mape','std_test_mape', 'rank_test_mape', 'mean_train_mape', 'std_train_mape']]

,params,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_test_meae,std_test_meae,rank_test_meae,mean_train_meae,std_train_meae,mean_test_r2,std_test_r2,rank_test_r2,mean_train_r2,std_train_r2,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_train_rmse,std_train_rmse,mean_test_mape,std_test_mape,rank_test_mape,mean_train_mape,std_train_mape
84,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 2...",-0.152194,0.014723,-0.108394,0.001861,7.184820e+07,9.763920e+06,103,5.095413e+07,941200.60534,0.700873,0.033118,4,0.874067,0.003085,3.184438e+08,1.927313e+07,105,1.993398e+08,2.270625e+06,0.217674,0.008665,105,0.146663,0.001712


### **Gradient Boosting Regression**

####**Sin tranformar la Y**

In [ ]:
pipe_gbt = Pipeline(steps = [('scaler', StandardScaler()), ('gbt', GradientBoostingRegressor(random_state=42))])

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'gbt__n_estimators':[70,90,100,120,180], 'gbt__max_depth':[3,5,10,12,15,20,25], 'gbt__min_samples_split':[5,10,15,17,25,30,40], 'gbt__min_samples_leaf':[1,3,5]}
para_grid = {'gbt__n_estimators':[100,120,150], 'gbt__max_depth':[6,7,8,9], 'gbt__min_samples_split':[4,5,6], 'gbt__min_samples_leaf':[2,3,4]}

modelo_gbt = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe_gbt, param_grid = para_grid )

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
metricas_gbt = metricas(modelo_gbt.best_estimator_,Y_train, modelo_gbt.predict(X_train), Y_test, modelo_gbt.predict(X_test))
metricas_gbt

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,([DecisionTreeRegressor(criterion='friedman_ms...,3.676256e+07,7.308039e+07,0.100509,0.215789,0.975973,0.77814,8.746738e+07,3.002781e+08


In [ ]:
modelo_gbt.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('gbt',
                 GradientBoostingRegressor(max_depth=7, min_samples_leaf=2,
                                           min_samples_split=4,
                                           random_state=42))])

####**Usando Y transformada**

**Búsqueda de hiperparámetros**

In [ ]:
# para_grid = {'gbt__n_estimators':[70,90,100,120,180], 'gbt__max_depth':[3,5,10,12,15,20,25], 'gbt__min_samples_split':[5,10,15,17,25,30,40], 'gbt__min_samples_leaf':[1,3,5]}
para_grid = {'gbt__n_estimators':[100,120,150], 'gbt__max_depth':[6,7,8,9], 'gbt__min_samples_split':[4,5,6], 'gbt__min_samples_leaf':[2,3,4]}
modelo_gbt_t = busqueda_hiperparametros(X_train, Y_train_t,var_stratify_train, pipe_gbt, param_grid = para_grid, transf=True )

Seccion Stratify
**para datos transformados**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
metricas_gbt_t = metricas(modelo_gbt_t.best_estimator_,Y_train, np.expm1(modelo_gbt_t.predict(X_train)), Y_test, np.expm1(modelo_gbt_t.predict(X_test)))
metricas_gbt_t

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,([DecisionTreeRegressor(criterion='friedman_ms...,1.729596e+07,7.487165e+07,0.049577,0.195328,0.989129,0.764,5.883480e+07,3.096988e+08


In [ ]:
modelo_gbt_t.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('gbt',
                 GradientBoostingRegressor(max_depth=9, min_samples_leaf=3,
                                           min_samples_split=4,
                                           n_estimators=120,
                                           random_state=42))])

### **XG Boost**

#### Sin transformar la Y

In [ ]:
import xgboost as xgb
# pipe_xgb = Pipeline(steps = [
#     ('scaler', StandardScaler()), 
#     ('xgb',  xgb.XGBRegressor(
#         objective='reg:squarederror', reg_alpha = 3, reg_lambda = 3, max_depth=5, learning_rate=0.05, n_jobs=-1, n_estimators = 175, 
#     ))
# ])

pipe_xgb = Pipeline(steps = [
    ('scaler', StandardScaler()), 
    ('xgb',  xgb.XGBRegressor(
        objective='reg:squarederror', n_jobs=-1, 
    ))
])

**Búsqueda de Hiperparámetros**

In [ ]:
para_grid = { 'xgb__max_depth':[5,6], 'xgb__learning_rate':[0.05,0.1], 'xgb__reg_alpha':[0.01,0.5,0.1]}
#'xgb__n_estimators':[100,120,150],
modelo_xgb = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe_xgb, param_grid = para_grid )

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
metricas_xgb = metricas(modelo_xgb.best_estimator_,Y_train, modelo_xgb.predict(X_train), Y_test, modelo_xgb.predict(X_test))
metricas_xgb

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"XGBRegressor(max_depth=5, n_jobs=-1, objective...",59268584.0,74319680.0,0.162174,0.210494,0.913513,0.782425,1.659486e+08,2.973642e+08


In [ ]:
modelo_xgb.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(max_depth=5, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

### **Red Neuronal**

####**Sin tranformar la Y**

In [ ]:
# from sklearn.neural_network import MLPRegressor
# pipe_rnn = Pipeline(steps = [('scaler', StandardScaler()), ('rnn', MLPRegressor(activation='relu',max_iter=100,random_state=42))])

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'rnn__hidden_layer_sizes': [(45,25),(95,50),(110,60),(150,75),(180,90)],
#     'rnn__alpha': [0.001,0.004,0.01,0.1],
#     'rnn__learning_rate_init': [0.001,0.004,0.01,0.1],
#     'rnn__max_iter':[200,500]}

# modelo_rnn = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe_rnn, param_grid = para_grid,transf=False )

In [ ]:
# metricas_rnn = metricas(modelo_rnn.best_estimator_,Y_train, modelo_rnn.predict(X_train), Y_test, modelo_rnn.predict(X_test))
# metricas_rnn

In [ ]:
# modelo_rnn.best_estimator_

####**Usando Y transformada**

In [ ]:
# para_grid = {'rnn__hidden_layer_sizes': [(45,25),(95,50),(110,60),(150,75),(180,90)],
#     'rnn__alpha': [0.001,0.004,0.01,0.1],
#     'rnn__learning_rate_init': [0.001,0.004,0.01,0.1],
#     'rnn__max_iter':[200,500]}
    
# modelo_rnn_t = busqueda_hiperparametros(X_train, Y_train_t,var_stratify_train, pipe_rnn, param_grid = para_grid, transf=True )

In [ ]:
# metricas_rnn_t = metricas(modelo_rnn_t.best_estimator_,Y_train, np.expm1(modelo_rnn_t.predict(X_train)), Y_test, np.expm1(modelo_rnn_t.predict(X_test)))
# metricas_rnn_t

In [ ]:
# modelo_rnn_t.best_estimator_

Referencia: https://www.cienciadedatos.net/documentos/py35-redes-neuronales-python.html

In [ ]:
# resultados_rnn =  pd.concat([metricas_rnn, metricas_rnn_t],ignore_index= True)
# metricas_rnn.to_csv('resultados_train_test_general_rnn.csv',sep=";",decimal=",")

KNN con Keras

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation

# model = Sequential([
#     Dense(32, input_shape=(input_dim,)),
#     Activation('tanh'),
#     Dense(nb_classes),
#     Activation('softmax'),
# ])

## Resumen

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/Seminario/Etapa Modelamiento/Resultados'

KeyboardInterrupt: ignored

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
cross_validation_rf = pd.DataFrame(modelo_rf.cv_results_)
cross_validation_rf = cross_validation_rf[cross_validation_rf['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_rf['escenario']='modelo_rf'

cross_validation_rft=pd.DataFrame(modelo_rf_t.cv_results_)
cross_validation_rft = cross_validation_rft[cross_validation_rft['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_rft['escenario']='modelo_rft'

cross_validation_gbt=pd.DataFrame(modelo_gbt.cv_results_)
cross_validation_gbt = cross_validation_gbt[cross_validation_gbt['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_gbt['escenario']='modelo_gbt'

cross_validation_gbtt=pd.DataFrame(modelo_gbt_t.cv_results_)
cross_validation_gbtt = cross_validation_gbtt[cross_validation_gbtt['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_gbtt['escenario']='modelo_gbtt'

cross_validation_xgb=pd.DataFrame(modelo_xgb.cv_results_)
cross_validation_xgb = cross_validation_xgb[cross_validation_xgb['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_xgb['escenario']='modelo_xgb'

# cross_validation_rnn=pd.DataFrame(modelo_rnn.cv_results_)
# cross_validation_rnn = cross_validation_rnn[cross_validation_rnn['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
# cross_validation_rnn['escenario']='modelo_rnn'

# cross_validation_rnnt=pd.DataFrame(modelo_rnn_t.cv_results_)
# cross_validation_rnnt = cross_validation_rnnt[cross_validation_rnnt['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
# cross_validation_rnnt['escenario']='modelo_rnnt'

# resultado_cross = pd.concat([cross_validation_rf,cross_validation_rft, cross_validation_gbt, cross_validation_gbtt,cross_validation_rnn,cross_validation_rnnt],
#                             ignore_index = True)
resultado_cross = pd.concat([cross_validation_rf,cross_validation_rft, cross_validation_gbt, cross_validation_gbtt,cross_validation_xgb],
                            ignore_index = True)

columna = pd.DataFrame(resultado_cross['params'])

resultado_cross.drop(['params'],axis=1, inplace=True)
resultado_cross = resultado_cross.drop_duplicates()
resultado_cross.reset_index(inplace=True, drop=False)
resultado_cross

,index,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_train_meae,mean_test_meae,mean_train_mape,mean_test_mape,mean_train_r2,mean_test_r2,mean_train_rmse,mean_test_rmse,escenario
0,0,-7.614567e+07,4.939623e+06,-5.863940e+07,7.721354e+05,5.863940e+07,7.614567e+07,0.168530,0.238447,0.889917,0.718674,3.473990e+16,9.546208e+16,modelo_rf
1,3,-1.521939e-01,1.472290e-02,-1.083944e-01,1.860813e-03,5.095413e+07,7.184820e+07,0.146663,0.217674,0.874067,0.700873,1.993398e+08,3.184438e+08,modelo_rft
2,4,-6.429997e+07,7.342527e+06,-3.451341e+07,1.022537e+06,3.451341e+07,6.429997e+07,0.093753,0.210542,0.979256,0.719483,6.547787e+15,9.521331e+16,modelo_gbt
3,5,-1.319696e-01,1.978878e-02,-3.026754e-02,2.505411e-03,1.575403e+07,5.986669e+07,0.045006,0.208542,0.990035,0.697333,5.591148e+07,3.208940e+08,modelo_gbtt
4,8,-7.209843e+07,9.336828e+06,-5.713852e+07,1.068107e+06,5.713852e+07,7.209843e+07,0.155515,0.219528,0.922193,0.741451,2.455513e+16,8.808756e+16,modelo_xgb
5,9,-7.209843e+07,9.336828e+06,-5.713852e+07,1.068107e+06,5.713852e+07,7.209843e+07,0.155515,0.219528,0.922193,0.741451,2.455513e+16,8.808756e+16,modelo_xgb
6,10,-7.209843e+07,9.336828e+06,-5.713852e+07,1.068107e+06,5.713852e+07,7.209843e+07,0.155515,0.219528,0.922193,0.741451,2.455513e+16,8.808756e+16,modelo_xgb


In [ ]:
resultado_metricas = pd.concat([metricas_rf, metricas_rf_t, metricas_gbt, metricas_gbt_t, metricas_xgb],
                            ignore_index = True)
resultado_metricas.drop_duplicates(inplace=True)
resultado_metricas.reset_index(inplace=True, drop=False)
resultado_metricas

,index,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.926453e+07,7.321584e+07,0.172622,0.215121,0.885647,0.778355,1.908200e+08,3.001326e+08
1,1,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.295492e+07,7.255825e+07,0.150728,0.205040,0.867465,0.754846,2.054302e+08,3.156481e+08
2,2,([DecisionTreeRegressor(criterion='friedman_ms...,3.676256e+07,7.308039e+07,0.100509,0.215789,0.975973,0.778140,8.746738e+07,3.002781e+08
3,3,([DecisionTreeRegressor(criterion='friedman_ms...,1.729596e+07,7.487165e+07,0.049577,0.195328,0.989129,0.764000,5.883480e+07,3.096988e+08
4,4,"XGBRegressor(max_depth=5, n_jobs=-1, objective...",5.926858e+07,7.431968e+07,0.162174,0.210494,0.913513,0.782425,1.659486e+08,2.973642e+08


In [ ]:
resultados =  pd.concat([resultado_metricas, resultado_cross], axis=1)
resultados

,index,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test,index,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_train_meae,mean_test_meae,mean_train_mape,mean_test_mape,mean_train_r2,mean_test_r2,mean_train_rmse,mean_test_rmse,escenario
0,0.0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.926453e+07,7.321584e+07,0.172622,0.215121,0.885647,0.778355,1.908200e+08,3.001326e+08,0,-7.614567e+07,4.939623e+06,-5.863940e+07,7.721354e+05,5.863940e+07,7.614567e+07,0.168530,0.238447,0.889917,0.718674,3.473990e+16,9.546208e+16,modelo_rf
1,1.0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.295492e+07,7.255825e+07,0.150728,0.205040,0.867465,0.754846,2.054302e+08,3.156481e+08,3,-1.521939e-01,1.472290e-02,-1.083944e-01,1.860813e-03,5.095413e+07,7.184820e+07,0.146663,0.217674,0.874067,0.700873,1.993398e+08,3.184438e+08,modelo_rft
2,2.0,([DecisionTreeRegressor(criterion='friedman_ms...,3.676256e+07,7.308039e+07,0.100509,0.215789,0.975973,0.778140,8.746738e+07,3.002781e+08,4,-6.429997e+07,7.342527e+06,-3.451341e+07,1.022537e+06,3.451341e+07,6.429997e+07,0.093753,0.210542,0.979256,0.719483,6.547787e+15,9.521331e+16,modelo_gbt
3,3.0,([DecisionTreeRegressor(criterion='friedman_ms...,1.729596e+07,7.487165e+07,0.049577,0.195328,0.989129,0.764000,5.883480e+07,3.096988e+08,5,-1.319696e-01,1.978878e-02,-3.026754e-02,2.505411e-03,1.575403e+07,5.986669e+07,0.045006,0.208542,0.990035,0.697333,5.591148e+07,3.208940e+08,modelo_gbtt
4,4.0,"XGBRegressor(max_depth=5, n_jobs=-1, objective...",5.926858e+07,7.431968e+07,0.162174,0.210494,0.913513,0.782425,1.659486e+08,2.973642e+08,8,-7.209843e+07,9.336828e+06,-5.713852e+07,1.068107e+06,5.713852e+07,7.209843e+07,0.155515,0.219528,0.922193,0.741451,2.455513e+16,8.808756e+16,modelo_xgb
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,-7.209843e+07,9.336828e+06,-5.713852e+07,1.068107e+06,5.713852e+07,7.209843e+07,0.155515,0.219528,0.922193,0.741451,2.455513e+16,8.808756e+16,modelo_xgb
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,-7.209843e+07,9.336828e+06,-5.713852e+07,1.068107e+06,5.713852e+07,7.209843e+07,0.155515,0.219528,0.922193,0.741451,2.455513e+16,8.808756e+16,modelo_xgb


In [ ]:
resultados.to_csv('resultados_train_test_general_varibles_basicas2.csv',sep=";",decimal=",")

In [ ]:
cross_validation_rf = pd.DataFrame(modelo_rf.cv_results_)
cross_validation_rf['escenario']='modelo_rf'

cross_validation_rft=pd.DataFrame(modelo_rf_t.cv_results_)
cross_validation_rft['escenario']='modelo_rft'

cross_validation_gbt=pd.DataFrame(modelo_gbt.cv_results_)
cross_validation_gbt['escenario']='modelo_gbt'

cross_validation_gbtt=pd.DataFrame(modelo_gbt_t.cv_results_)
cross_validation_gbtt['escenario']='modelo_gbtt'

cross_validation_xgb=pd.DataFrame(modelo_xgb.cv_results_)
cross_validation_xgb['escenario']='modelo_xgb'

resultado_cross_total = pd.concat([cross_validation_rf,cross_validation_rft, cross_validation_gbt, cross_validation_gbtt,cross_validation_xgb],
                            ignore_index = True)

columna = pd.DataFrame(resultado_cross_total['params'])

resultado_cross_total.drop(['params'],axis=1, inplace=True)
resultado_cross_total = resultado_cross_total.drop_duplicates()
resultado_cross_total.reset_index(inplace=True, drop=False)

In [ ]:
resultado_cross_total.to_csv('resultados_crossvalidacion_general_varibles_basicas2.csv',sep=";",decimal=",")

##Exportar modelos

In [38]:
from joblib import dump, load
import xgboost as xgb

In [39]:
#Exportar modelo básico 15 variables

data_model=df_propiedades.copy()
varibles_considerar = ['tipo','precio','area_m2','banos','garajes','antiguedad','estrato','ubicacion','tipo_propiedad','balcon','zonas_verdes','en_conjunto_cerrado','zona_infantil','supermercados_ccomerciales','colegios_universidades','trans_publico_cercano','habitaciones']
data_model = data_model.loc[:,varibles_considerar]

X = data_model.drop(['precio'], axis=1)
y = data_model['precio']

enc = OneHotEncoder(handle_unknown='ignore')
data_aux = pd.DataFrame(enc.fit_transform(X[['tipo','tipo_propiedad','ubicacion','antiguedad']]).toarray(), columns= enc.get_feature_names(['tipo','tipo_propiedad','ubicacion','antiguedad']))
data_aux['fila'] = range(0, X.shape[0])

X.drop(['tipo','tipo_propiedad','antiguedad'],axis=1,inplace=True)
X['fila'] = range(0, X.shape[0])

X = pd.merge(data_aux,X, on = 'fila', how='inner')
X.drop(['fila'], axis=1, inplace=True)

X_train, X_test, Y_train, Y_test, Y_train_t, Y_test_t, var_stratify_train = div_train_test(X, y, var_stratify = 'ubicacion')

pipe_xgb = Pipeline(steps = [('scaler', StandardScaler()), ('xgb', xgb.XGBRegressor(max_depth=7, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

pipe_xgb.fit(X_train, Y_train)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(max_depth=7, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

In [40]:
metricas_ = metricas(pipe_xgb,Y_train, pipe_xgb.predict(X_train), Y_test, pipe_xgb.predict(X_test))
metricas_

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"XGBRegressor(max_depth=7, n_jobs=-1, objective...",39303600.0,83417216.0,0.109669,0.208713,0.967241,0.789518,1.021327e+08,2.924766e+08


In [41]:
dump(pipe_gbt, 'export_model_xgb_15112021.joblib') 

['export_model_xgb_15112021.joblib']

In [42]:
X_train.head(1).to_csv('x_labels_xgb_15V.csv')

In [43]:
#Exportar modelo 7 variables

data_model=df_propiedades.copy()
varibles_considerar = ['tipo','tipo_propiedad','precio','area_m2','banos','estrato','habitaciones','ubicacion']


data_model = data_model.loc[:,varibles_considerar]

X = data_model.drop(['precio'], axis=1)
y = data_model['precio']

In [44]:
enc = OneHotEncoder(handle_unknown='ignore')
data_aux = pd.DataFrame(enc.fit_transform(X[['tipo','tipo_propiedad','ubicacion']]).toarray(), columns= enc.get_feature_names(['tipo','tipo_propiedad','ubicacion']))
data_aux['fila'] = range(0, X.shape[0])

X.drop(['tipo','tipo_propiedad'],axis=1,inplace=True)
X['fila'] = range(0, X.shape[0])

X = pd.merge(data_aux,X, on = 'fila', how='inner')
X.drop(['fila'], axis=1, inplace=True)

X_train, X_test, Y_train, Y_test, Y_train_t, Y_test_t, var_stratify_train = div_train_test(X, y, var_stratify = 'ubicacion')

pipe_xgb2 = Pipeline(steps = [('scaler', StandardScaler()), ('xgb', xgb.XGBRegressor(max_depth=7, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

pipe_xgb2.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(max_depth=7, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

In [45]:
pipe_gbt2.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(max_depth=7, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

In [46]:
metricas_2 = metricas(pipe_xgb2,Y_train, pipe_xgb2.predict(X_train), Y_test, pipe_xgb2.predict(X_test))
metricas_2

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"XGBRegressor(max_depth=7, n_jobs=-1, objective...",45078528.0,80152320.0,0.128365,0.214738,0.949141,0.760602,1.272574e+08,3.119207e+08


In [47]:
dump(pipe_xgb2, 'export_model_xgb_7V_15112021.joblib') 

['export_model_xgb_7V_15112021.joblib']

In [48]:
X_train.head(1).to_csv('x_labels_xgb_7V.csv')